### Bank Customer Product Recommendation System

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import datetime as dt
import random

# Data Viz 
import seaborn as sns
import matplotlib.pyplot as plt

# Data Manipulation
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import MinMaxScaler

# Similarity calculation
from sklearn.metrics.pairwise import cosine_similarity

# settings
pd.options.display.max_rows = 100
pd.options.display.max_columns = None

# Math
import math

# Remove warnings
import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### 1. Data Import and Profiling

The data set is stored within the working folder. In real life workplace, most of user and items data should be saved and updated either in the operational databases of the front-end systems such as CRM, Trading Desks, or in Enterprise Data Warehousr (EDW) that extracts and integrates data from many different systems and consolidate them into a single repository through ETL. 

In [2]:
# import the tables
train = pd.read_csv(filepath_or_buffer='/train_ver2.csv.zip')
test = pd.read_csv(filepath_or_buffer='/test_ver2.csv.zip')
prob = pd.read_csv(filepath_or_buffer='/probabilities_85_avg.csv')
bycus = pd.read_csv(filepath_or_buffer='/submission.csv')

In [3]:
## Data Profiling
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929615 entries, 0 to 929614
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   fecha_dato             929615 non-null  object 
 1   ncodpers               929615 non-null  int64  
 2   ind_empleado           929615 non-null  object 
 3   pais_residencia        929615 non-null  object 
 4   sexo                   929610 non-null  object 
 5   age                    929615 non-null  int64  
 6   fecha_alta             929615 non-null  object 
 7   ind_nuevo              929615 non-null  int64  
 8   antiguedad             929615 non-null  int64  
 9   indrel                 929615 non-null  int64  
 10  ult_fec_cli_1t         1683 non-null    object 
 11  indrel_1mes            929592 non-null  float64
 12  tiprel_1mes            929592 non-null  object 
 13  indresi                929615 non-null  object 
 14  indext                 929615 non-nu

In [4]:
test.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
fecha_dato,929615,1,2016-06-28,929615,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ncodpers,929615.0,NaN,NaN,NaN,879456.621092,448156.939785,15889.0,483361.5,966425.0,1264316.5,1553689.0
ind_empleado,929615,5,N,929096,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pais_residencia,929615,118,ES,925619,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sexo,929610,2,V,504515,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age,929615.0,NaN,NaN,NaN,40.249821,17.185119,2.0,25.0,39.0,51.0,164.0
fecha_alta,929615,6750,2014-07-28,3421,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ind_nuevo,929615.0,NaN,NaN,NaN,0.027849,0.164541,0.0,0.0,0.0,0.0,1.0
antiguedad,929615.0,NaN,NaN,NaN,77.728413,1797.824518,-999999.0,23.0,55.0,136.0,257.0
indrel,929615.0,NaN,NaN,NaN,1.177422,4.166039,1.0,1.0,1.0,1.0,99.0


In [5]:
# Check missing values
test.isnull().sum()/test.shape[0] * 100

fecha_dato                0.000000
ncodpers                  0.000000
ind_empleado              0.000000
pais_residencia           0.000000
sexo                      0.000538
age                       0.000000
fecha_alta                0.000000
ind_nuevo                 0.000000
antiguedad                0.000000
indrel                    0.000000
ult_fec_cli_1t           99.818957
indrel_1mes               0.002474
tiprel_1mes               0.002474
indresi                   0.000000
indext                    0.000000
conyuemp                 99.988813
canal_entrada             0.223856
indfall                   0.000000
tipodom                   0.000000
cod_prov                  0.429855
nomprov                   0.429855
ind_actividad_cliente     0.000000
renta                     0.000000
segmento                  0.241821
dtype: float64

In [6]:
## Inspect the data sample
test.sample(5)

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
248947,2016-06-28,1478840,N,ES,H,21,2015-10-14,0,8,1,NaN,1.0,I,S,N,NaN,KHQ,N,1,50.0,ZARAGOZA,0,NA,03 - UNIVERSITARIO
184486,2016-06-28,1081078,N,ES,H,25,2012-10-15,0,44,1,NaN,1.0,I,S,N,NaN,KHE,N,1,21.0,HUELVA,0,NA,03 - UNIVERSITARIO
702650,2016-06-28,852790,N,ES,V,55,2009-09-25,0,81,1,NaN,1.0,A,S,N,NaN,KFC,N,1,15.0,"CORUÑA, A",1,NA,02 - PARTICULARES
594024,2016-06-28,85221,N,ES,H,57,1998-02-09,0,220,1,NaN,1.0,I,S,N,NaN,KAT,N,1,8.0,BARCELONA,0,207773.34,02 - PARTICULARES
240240,2016-06-28,1466232,N,ES,H,26,2015-10-01,0,9,1,NaN,1.0,I,S,N,NaN,KHQ,N,1,8.0,BARCELONA,0,NA,03 - UNIVERSITARIO


From the brief summary, the feature names are not pretty intuitive, and there are missing values exist in some fields such as "ult_fec_cli_1t", "conyuemp", "renta". Besides, the data types of some features like "age" and "fecha_alta" are not correct and should be adjusted.

#### 2. Data Cleaning

In this step, the cleaning process will include the following parts:

1) Column Names Conversion
    
2) Data Type Correction
    
3) Missing values imputation
    
4) Outliers Correction

In [7]:
test.columns

Index(['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
       'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
       'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
       'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'cod_prov',
       'nomprov', 'ind_actividad_cliente', 'renta', 'segmento'],
      dtype='object')

In [8]:
# 1) Feature Name Transformation
col_names = {"ncodpers":"cust_id", "ind_empleado":"emp_index","pais_residencia":"residence",
            "sexo":"sex","fecha_alta":"first_date","ind_nuevo":"new_cust","antiguedad":"seniority",
            "indrel":"is_primary","ult_fec_cli_1t":"last_primary_date","indrel_1mes":"cust_type",
            "tiprel_1mes":"cust_rel_type","indresi":"residence_index","indext":"foreigner_index",
            "conyuemp":"spouse_index","canal_entrada":"channel","cod_prov":"province","nomprov":"province_name",
            "ind_actividad_cliente":"active_index","renta":"income","segmento":"segment"}

col_names1 = {"ncodpers":"cust_id"}
train.rename(col_names, axis = 1, inplace = True)
test.rename(col_names, axis = 1, inplace = True)
prob.rename(col_names1, axis = 1, inplace = True)
bycus.rename(col_names1, axis = 1, inplace = True)

In [9]:
# 2) Data Type Conversion
# Convert the features into their intuitive types
train.age = pd.to_numeric(train.age, errors='coerce')
train.income = pd.to_numeric(train.income, errors='coerce')
train.seniority = pd.to_numeric(train.seniority, errors='coerce')
train.first_date = pd.to_datetime(train.first_date, errors = 'coerce')
train['fecha_dato'] = pd.to_datetime(train['fecha_dato'])

test.age = pd.to_numeric(test.age, errors='coerce')
test.income = pd.to_numeric(test.income, errors='coerce')
test.seniority = pd.to_numeric(test.seniority, errors='coerce')
test.first_date = pd.to_datetime(test.first_date, errors = 'coerce')

test['fecha_dato'] = pd.to_datetime(test['fecha_dato'])

In [10]:
# 3) Missing values imputation

# For features with missing values percentage < 10%, use the most common factor level to impute
cols = ['emp_index','residence','sex','first_date','new_cust','is_primary',"cust_type","cust_rel_type",
       "province","province_name","active_index","channel","segment"]

for i in cols:
    train.loc[train[i].isnull(), i] = train[i].value_counts().index[0]
    test.loc[test[i].isnull(), i] = test[i].value_counts().index[0]
    
# For features with missing value accounts for over 10%, impute their missing values based on the mean at different dimensions
# For Income, impute the missings using the medians of Customer Segment and Province
train['income'].fillna(train['income'].mean(), inplace = True)
test['income'].fillna(test['income'].mean(), inplace = True)

In [11]:
# Drop the last primary date and spouse index fields given over 99% missing values
train.drop(['last_primary_date','spouse_index'], axis = 1, inplace = True)
test.drop(['last_primary_date','spouse_index'], axis = 1, inplace = True)

In [12]:
# Filter rows where 'date' is greater than or equal to '2016-01-01'
filtered_df = test[test['first_date'] >= '2015-01-01']
test = filtered_df

In [13]:
prob = prob[prob['cust_id'].isin(test['cust_id'])]

In [14]:
# Add 7 years and 6 months to 'first_date' to create 'new_date'
train['new_first_date'] = train['first_date'] + pd.DateOffset(years=7, months=6)
test['new_first_date'] = test['first_date'] + pd.DateOffset(years=7, months=6)

In [15]:
del train['first_date']
del test['first_date']

In [16]:
# Take a look at the cleaned data
test.head()

,fecha_dato,cust_id,emp_index,residence,sex,age,new_cust,seniority,is_primary,cust_type,cust_rel_type,residence_index,foreigner_index,channel,indfall,tipodom,province,province_name,active_index,income,segment,new_first_date
105,2016-06-28,1170620,N,ES,V,39,1,5,1,1.0,A,S,N,KFC,N,1,18.0,GRANADA,1,134087.870595,02 - PARTICULARES,2023-07-22
671,2016-06-28,1169651,N,ES,V,67,0,34,1,1.0,A,S,S,KAT,N,1,8.0,BARCELONA,0,134087.870595,02 - PARTICULARES,2023-07-22
674,2016-06-28,1169655,N,ES,V,52,0,15,1,1.0,A,S,N,KAT,N,1,37.0,SALAMANCA,1,81011.520000,02 - PARTICULARES,2022-09-10
1351,2016-06-28,1172024,N,ES,H,42,1,5,1,1.0,A,S,N,KHM,N,1,8.0,BARCELONA,1,134087.870595,02 - PARTICULARES,2023-07-12
1585,2016-06-28,1171168,N,ES,V,31,0,34,1,1.0,A,S,S,KFC,N,1,38.0,SANTA CRUZ DE TENERIFE,1,134087.870595,02 - PARTICULARES,2023-10-04


#### 3. Feature Engineering

As the most important step in the project, the quality of feature engineering determines the overall performance of product recommendation. Part of engineering is usually done using domain knowledge (e.g. which features are considered more impactful and intuitive in the industry) and previous experiences. A good feature engineering is not only helpful for the modeling learning process, but also make the raw features more understandable and intuitive to the ending users and business partners.

In [17]:
# 1) Age group: group the age for lifestages: young, adult, senior, elder
age_group = [0, 20, 45, 65, 100]
age_labels = ['young', 'adult', 'senior', 'elder']

train['age_grouped'] = pd.cut(train.age, bins = age_group, labels = age_labels)
test['age_grouped'] = pd.cut(test.age, bins = age_group, labels = age_labels)

In [18]:
# Change values in the 'sex' column
train['sex'] = train['sex'].replace({'H': 'M', 'V': 'F'})
test['sex'] = test['sex'].replace({'H': 'M', 'V': 'F'})

In [19]:
# Change all values in the 'residence' column to 'VN'
train['residence'] = 'VN'
test['residence'] = 'VN'

In [20]:
# Change values in the 'cust_rel_type'
train['cust_rel_type'] = train['cust_rel_type'].replace({'A': 'Active', 'I': 'Inactive'})
test['cust_rel_type'] = test['cust_rel_type'].replace({'A': 'Active', 'I': 'Inactive'})

In [21]:
unique_values1 = train['province_name'].unique()
unique_values2 = test['province_name'].unique()

In [22]:
print(unique_values1)
print(unique_values2)

['MALAGA' 'CIUDAD REAL' 'ZARAGOZA' 'TOLEDO' 'LEON' 'GIPUZKOA' 'CACERES'
 'GIRONA' 'ZAMORA' 'BARCELONA' 'SALAMANCA' 'BURGOS' 'HUESCA' 'NAVARRA'
 'AVILA' 'SEGOVIA' 'LUGO' 'LERIDA' 'MADRID' 'ALICANTE' 'SORIA' 'SEVILLA'
 'CANTABRIA' 'BALEARS, ILLES' 'VALLADOLID' 'PONTEVEDRA' 'VALENCIA'
 'TERUEL' 'CORUÑA, A' 'OURENSE' 'JAEN' 'CUENCA' 'BIZKAIA' 'CASTELLON'
 'RIOJA, LA' 'ALBACETE' 'BADAJOZ' 'MURCIA' 'CADIZ' 'ALMERIA' 'GUADALAJARA'
 'PALENCIA' 'PALMAS, LAS' 'CORDOBA' 'HUELVA' 'GRANADA' 'ASTURIAS'
 'SANTA CRUZ DE TENERIFE' 'MELILLA' 'TARRAGONA' 'ALAVA' 'CEUTA']
['GRANADA' 'BARCELONA' 'SALAMANCA' 'SANTA CRUZ DE TENERIFE' 'MURCIA'
 'MADRID' 'MALAGA' 'VALLADOLID' 'TOLEDO' 'VALENCIA' 'SEVILLA' 'GIPUZKOA'
 'GIRONA' 'ALICANTE' 'JAEN' 'CUENCA' 'CORUÑA, A' 'ZARAGOZA' 'PALENCIA'
 'GUADALAJARA' 'ASTURIAS' 'BURGOS' 'ALAVA' 'HUELVA' 'CADIZ' 'CIUDAD REAL'
 'CORDOBA' 'CASTELLON' 'BADAJOZ' 'CANTABRIA' 'ALBACETE' 'CACERES'
 'BALEARS, ILLES' 'ALMERIA' 'OURENSE' 'PALMAS, LAS' 'BIZKAIA' 'LEON'
 'ZAMORA' 'PONTEVED

In [23]:
frequency1 = train['province_name'].value_counts()
frequency2 = test['province_name'].value_counts()

In [24]:
print(frequency1)
print(frequency2)

MADRID                    4503191
BARCELONA                 1275219
VALENCIA                   682304
SEVILLA                    605164
CORUÑA, A                  429322
MURCIA                     396759
MALAGA                     367023
ZARAGOZA                   342543
ALICANTE                   313397
CADIZ                      294684
PONTEVEDRA                 280026
ASTURIAS                   265749
VALLADOLID                 238259
PALMAS, LAS                235700
BADAJOZ                    192996
BIZKAIA                    185888
TOLEDO                     183067
GRANADA                    178562
SALAMANCA                  164238
CANTABRIA                  155706
CORDOBA                    144679
CACERES                    129912
BALEARS, ILLES             124933
HUELVA                     122283
CIUDAD REAL                119390
ALBACETE                   114128
CASTELLON                  104295
TARRAGONA                  100115
BURGOS                      97188
GIRONA        

In [25]:
# Change values in the 'province_name' column
train['province_name'] = train['province_name'].replace({
    'MADRID': 'Ho Chi Minh City',
    'BARCELONA': 'Hanoi',
    'VALENCIA': 'Thanh Hoa',
    'SEVILLA': 'Nghe An',
    'CORUÑA, A': 'Dong Nai',
    'MURCIA': 'Binh Duong',
    'MALAGA': 'Hai Phong',
    'ZARAGOZA': 'An Giang',
    'ALICANTE': 'Hai Duong',
    'CADIZ': 'Dak Lak',
    'PONTEVEDRA': 'Thai Binh',
    'ASTURIAS': 'Bac Giang',
    'VALLADOLID': 'Nam Dinh',
    'PALMAS, LAS': 'Tien Giang',
    'BADAJOZ': 'Kien Giang',
    'BIZKAIA': 'Long An',
    'TOLEDO': 'Dong Thap',
    'GRANADA': 'Gia Lai',
    'SALAMANCA': 'Quang Nam',
    'CANTABRIA': 'Binh Dinh',
    'CORDOBA': 'Phu Tho',
    'CACERES': 'Bac Ninh',
    'BALEARS, ILLES': 'Quang Ninh',
    'HUELVA': 'Lam Dong',
    'CIUDAD REAL': 'Ha Tinh',
    'ALBACETE': 'Ben Tre',
    'CASTELLON': 'Thai Nguyen',
    'TARRAGONA': 'Hung Yen',
    'BURGOS': 'Son La',
    'GIRONA': 'Can Tho',
    'NAVARRA': 'Quang Ngai',
    'RIOJA, LA': 'Khanh Hoa',
    'LUGO': 'Binh Thuan',
    'OURENSE': 'Soc Trang',
    'LEON': 'Ca Mau',
    'LERIDA': 'Tay Ninh',
    'GIPUZKOA': 'Vinh Phuc',
    'SANTA CRUZ DE TENERIFE': 'Ba Ria – Vung Tau',
    'GUADALAJARA': 'Đa Nang',
    'JAEN': 'Thua Thien Hue',
    'ALMERIA': 'Vinh Long',
    'CUENCA': 'Tra Vinh',
    'ZAMORA': 'Binh Phuoc',
    'PALENCIA': 'Ninh Binh',
    'SEGOVIA': 'Phu Yen',
    'HUESCA': 'Bac Lieu',
    'AVILA': 'Quang Binh',
    'ALAVA': 'Ha Giang',
    'TERUEL': 'Hoa Binh',
    'SORIA': 'Ha Nam',
    'MELILLA': 'Yen Bai',
    'CEUTA': 'Tuyen Quang'})

In [26]:
# Change values in the 'province_name' column
test['province_name'] = test['province_name'].replace({
    'MADRID': 'Ho Chi Minh City',
    'BARCELONA': 'Hanoi',
    'VALENCIA': 'Thanh Hoa',
    'SEVILLA': 'Nghe An',
    'CORUÑA, A': 'Dong Nai',
    'MURCIA': 'Binh Duong',
    'MALAGA': 'Hai Phong',
    'ZARAGOZA': 'An Giang',
    'ALICANTE': 'Hai Duong',
    'CADIZ': 'Dak Lak',
    'PONTEVEDRA': 'Thai Binh',
    'ASTURIAS': 'Bac Giang',
    'VALLADOLID': 'Nam Dinh',
    'PALMAS, LAS': 'Tien Giang',
    'BADAJOZ': 'Kien Giang',
    'BIZKAIA': 'Long An',
    'TOLEDO': 'Dong Thap',
    'GRANADA': 'Gia Lai',
    'SALAMANCA': 'Quang Nam',
    'CANTABRIA': 'Binh Dinh',
    'CORDOBA': 'Phu Tho',
    'CACERES': 'Bac Ninh',
    'BALEARS, ILLES': 'Quang Ninh',
    'HUELVA': 'Lam Dong',
    'CIUDAD REAL': 'Ha Tinh',
    'ALBACETE': 'Ben Tre',
    'CASTELLON': 'Thai Nguyen',
    'TARRAGONA': 'Hung Yen',
    'BURGOS': 'Son La',
    'GIRONA': 'Can Tho',
    'NAVARRA': 'Quang Ngai',
    'RIOJA, LA': 'Khanh Hoa',
    'LUGO': 'Binh Thuan',
    'OURENSE': 'Soc Trang',
    'LEON': 'Ca Mau',
    'LERIDA': 'Tay Ninh',
    'GIPUZKOA': 'Vinh Phuc',
    'SANTA CRUZ DE TENERIFE': 'Ba Ria – Vung Tau',
    'GUADALAJARA': 'Đa Nang',
    'JAEN': 'Thua Thien Hue',
    'ALMERIA': 'Vinh Long',
    'CUENCA': 'Tra Vinh',
    'ZAMORA': 'Binh Phuoc',
    'PALENCIA': 'Ninh Binh',
    'SEGOVIA': 'Phu Yen',
    'HUESCA': 'Bac Lieu',
    'AVILA': 'Quang Binh',
    'ALAVA': 'Ha Giang',
    'TERUEL': 'Hoa Binh',
    'SORIA': 'Ha Nam',
    'MELILLA': 'Yen Bai',
    'CEUTA': 'Tuyen Quang'})

In [27]:
train.rename(columns={
    'ind_ahor_fin_ult1': 'green_bonds',
    'ind_aval_fin_ult1': 'insurance',
    'ind_cco_fin_ult1': 'business_household_loans',
    'ind_cder_fin_ult1': 'business_loans',
    'ind_cno_fin_ult1': 'mutual_funds',
    'ind_ctju_fin_ult1': 'credit_card',
    'ind_ctma_fin_ult1': 'bonds',
    'ind_ctop_fin_ult1': 'mortgage',
    'ind_ctpp_fin_ult1': 'saving_account',
    'ind_deco_fin_ult1': 'deposits',
    'ind_deme_fin_ult1': 'foreign_curency_exchange_services',
    'ind_dela_fin_ult1': 'long_term_deposits',
    'ind_ecue_fin_ult1': 'auto_loans',
    'ind_fond_fin_ult1': 'bill_pay_services',
    'ind_hip_fin_ult1': 'wealth_management',
    'ind_plan_fin_ult1': 'corporate_credit_cards',
    'ind_pres_fin_ult1': 'current_accounts',
    'ind_reca_fin_ult1': 'equipment_financing',
    'ind_tjcr_fin_ult1': 'stocks',
    'ind_valo_fin_ult1': 'Mobile_banking',
    'ind_viv_fin_ult1': 'student_loans',
    'ind_nomina_ult1': 'payroll_account',
    'ind_nom_pens_ult1': 'retirement_account',
    'ind_recibo_ult1': 'debit_cards'
}, inplace=True)

In [28]:
test.rename(columns={
    'ind_ahor_fin_ult1': 'green_bonds',
    'ind_aval_fin_ult1': 'insurance',
    'ind_cco_fin_ult1': 'business_household_loans',
    'ind_cder_fin_ult1': 'business_loans',
    'ind_cno_fin_ult1': 'mutual_funds',
    'ind_ctju_fin_ult1': 'credit_card',
    'ind_ctma_fin_ult1': 'bonds',
    'ind_ctop_fin_ult1': 'mortgage',
    'ind_ctpp_fin_ult1': 'saving_account',
    'ind_deco_fin_ult1': 'deposits',
    'ind_deme_fin_ult1': 'foreign_curency_exchange_services',
    'ind_dela_fin_ult1': 'long_term_deposits',
    'ind_ecue_fin_ult1': 'auto_loans',
    'ind_fond_fin_ult1': 'bill_pay_services',
    'ind_hip_fin_ult1': 'wealth_management',
    'ind_plan_fin_ult1': 'corporate_credit_cards',
    'ind_pres_fin_ult1': 'current_accounts',
    'ind_reca_fin_ult1': 'equipment_financing',
    'ind_tjcr_fin_ult1': 'stocks',
    'ind_valo_fin_ult1': 'Mobile_banking',
    'ind_viv_fin_ult1': 'student_loans',
    'ind_nomina_ult1': 'payroll_account',
    'ind_nom_pens_ult1': 'retirement_account',
    'ind_recibo_ult1': 'debit_cards'
}, inplace=True)

In [29]:
prob.rename(columns={
    'ind_ahor_fin_ult1_predict': 'green_bonds',
    'ind_aval_fin_ult1_predict': 'insurance',
    'ind_cco_fin_ult1_predict': 'business_household_loans',
    'ind_cder_fin_ult1_predict': 'business_loans',
    'ind_cno_fin_ult1_predict': 'mutual_funds',
    'ind_ctju_fin_ult1_predict': 'credit_card',
    'ind_ctma_fin_ult1_predict': 'bonds',
    'ind_ctop_fin_ult1_predict': 'mortgage',
    'ind_ctpp_fin_ult1_predict': 'saving_account',
    'ind_deco_fin_ult1_predict': 'deposits',
    'ind_deme_fin_ult1_predict': 'foreign_curency_exchange_services',
    'ind_dela_fin_ult1_predict': 'long_term_deposits',
    'ind_ecue_fin_ult1_predict': 'auto_loans',
    'ind_fond_fin_ult1_predict': 'bill_pay_services',
    'ind_hip_fin_ult1_predict': 'wealth_management',
    'ind_plan_fin_ult1_predict': 'corporate_credit_cards',
    'ind_pres_fin_ult1_predict': 'current_accounts',
    'ind_reca_fin_ult1_predict': 'equipment_financing',
    'ind_tjcr_fin_ult1_predict': 'stocks',
    'ind_valo_fin_ult1_predict': 'Mobile_banking',
    'ind_viv_fin_ult1_predict': 'student_loans',
    'ind_nomina_ult1_predict': 'payroll_account',
    'ind_nom_pens_ult1_predict': 'retirement_account',
    'ind_recibo_ult1_predict': 'debit_cards'
}, inplace=True)

In [30]:
# Change values in the 'emp_index' column
train['emp_index'] = train['emp_index'].replace({'S': 'Y'})
test['emp_index'] = test['emp_index'].replace({'S': 'Y'})

In [31]:
# Change values in the 'foreigner_index' column
train['foreigner_index'] = train['foreigner_index'].replace({'S': 'Y'})
test['foreigner_index'] = test['foreigner_index'].replace({'S': 'Y'})

In [32]:
# Change values in the 'residence_index' column
train['residence_index'] = train['residence_index'].replace({'S': 'Y'})
test['residence_index'] = test['residence_index'].replace({'S': 'Y'})

In [33]:
# Change values in the 'segment' column
train['segment'] = train['segment'].replace({'01 - TOP':'high school diploma','02 - PARTICULARES': 'bachelor degree','03 - UNIVERSITARIO':'advanced degree'})
test['segment'] = test['segment'].replace({'01 - TOP':'high school diploma','02 - PARTICULARES': 'bachelor degree','03 - UNIVERSITARIO':'advanced degree'})

In [34]:
#Customer type: Priority & Normal 
train['is_primary'] = train['is_primary'].replace({'1':'PIORITY','99': 'STANDARD'})
test['is_primary'] = test['is_primary'].replace({'1':'PIORITY','99': 'STANDARD'})

In [35]:
channel_frequency = test['channel'].value_counts()
print(channel_frequency)

KHQ    74727
KHM    31692
KHN    16433
KHK    10523
RED     2748
KHL     2499
KFC     2121
KAT     1375
KHO     1318
KFA      811
KHE      161
KHF      156
KHP       85
KHS       21
007       15
KAG       10
KHD       10
013        7
KAZ        7
KAA        6
KAS        5
KAY        4
KHC        3
KCC        3
KAW        2
KAJ        2
KFD        2
KAQ        2
KFL        2
KBZ        2
KAF        2
KEY        1
KBW        1
KAH        1
KFF        1
KAB        1
KCB        1
KCI        1
KFP        1
KCK        1
KAR        1
KCH        1
KAE        1
Name: channel, dtype: int64


In [36]:
#Customer acquisition channel 
channel_dict = {
    'KFC':'Open New Branches', 
    'KAT':'Display Ads', 
    'KHM':'Open New Branches', 
    'KHN':'Marketing Campaign 1', 
    'KFA':'Mobile Apps and Websites', 
    'KHO':'Open New Branches', 
    'KHL':'Partnerships', 
    'RED':'Paid Search', 
    'KHK':'Email Campaigns',
    'KHQ':'Social Media Campaign', 
    'KHC':'Marketing Campaign 2', 
    'KHE':'Community Events', 
    'KAZ':'Marketing Campaign 3', 
    'KHP':'Referral Programs', 
    'KHD':'Branch', 
    'KHS':'Branch', 
    'KHF':'Branch', 
    'KAG':'Branch',
    'KBZ':'Branch', 
    '007':'Branch', 
    'KAF':'Branch', 
    '013':'Branch', 
    'KCK':'Branch', 
    'KCB':'Branch', 
    'KAY':'Branch', 
    'KCC':'Branch', 
    'KAB':'Branch',
    'KAH':'Branch', 
    'KAA':'Branch', 
    'KEY':'Branch', 
    'KAS':'Branch', 
    'KBW':'Branch', 
    'KFL':'Branch', 
    'KFF':'Branch', 
    'KAQ':'Branch', 
    'KFD':'Branch',
    'KCI':'Branch', 
    'KFP':'Branch', 
    'KAJ':'Branch', 
    'KAR':'Branch', 
    'KAW':'Community Events', 
    'KCH':'Community Events', 
    'KAE':'Community Events'
}
train['channel'] = train['channel'].replace(channel_dict)
test['channel'] = test['channel'].replace(channel_dict)

In [37]:
occupations = [
    'corporations',
    'business household',
    'businessman',
    'office worker',
    'lawyer',
    'doctor',
    'public official',
    'teacher',
    'engineer',
    'artist',
    'scientist',
    'chef',
    'writer',
    'pilot',
    'nurse',
    'musician',
    'designer',
    'athlete',
    'journalist',
]

In [38]:
test['Occupation'] = random.choices(occupations, k=len(test))

In [39]:
marital_status = ['Single','Married']

In [40]:
test['Marital status'] = random.choices(marital_status, k=len(test))

In [41]:
test.head()

,fecha_dato,cust_id,emp_index,residence,sex,age,new_cust,seniority,is_primary,cust_type,cust_rel_type,residence_index,foreigner_index,channel,indfall,tipodom,province,province_name,active_index,income,segment,new_first_date,age_grouped,Occupation,Marital status
105,2016-06-28,1170620,N,VN,F,39,1,5,1,1.0,Active,Y,N,Open New Branches,N,1,18.0,Gia Lai,1,134087.870595,bachelor degree,2023-07-22,adult,lawyer,Single
671,2016-06-28,1169651,N,VN,F,67,0,34,1,1.0,Active,Y,Y,Display Ads,N,1,8.0,Hanoi,0,134087.870595,bachelor degree,2023-07-22,elder,lawyer,Married
674,2016-06-28,1169655,N,VN,F,52,0,15,1,1.0,Active,Y,N,Display Ads,N,1,37.0,Quang Nam,1,81011.520000,bachelor degree,2022-09-10,senior,scientist,Single
1351,2016-06-28,1172024,N,VN,M,42,1,5,1,1.0,Active,Y,N,Open New Branches,N,1,8.0,Hanoi,1,134087.870595,bachelor degree,2023-07-12,adult,journalist,Married
1585,2016-06-28,1171168,N,VN,F,31,0,34,1,1.0,Active,Y,Y,Open New Branches,N,1,38.0,Ba Ria – Vung Tau,1,134087.870595,bachelor degree,2023-10-04,adult,doctor,Single


### Recommend top 7 products by customer

In [42]:
bycus.head()

,cust_id,added_products
0,15889,ind_recibo_ult1 ind_nom_pens_ult1 ind_nomina_u...
1,1170544,ind_recibo_ult1 ind_nom_pens_ult1 ind_nomina_u...
2,1170545,ind_recibo_ult1 ind_nom_pens_ult1 ind_nomina_u...
3,1170547,ind_recibo_ult1 ind_nom_pens_ult1 ind_nomina_u...
4,1170548,ind_recibo_ult1 ind_nom_pens_ult1 ind_nomina_u...


In [43]:
# Split the 'added_products' column into 7 columns
bycus[['product1','product2','product3','product4','product5', 'product6', 'product7']] = bycus['added_products'].str.split(' ', n=7, expand=True)

In [44]:
bycus.sample(5)

,cust_id,added_products,product1,product2,product3,product4,product5,product6,product7
728631,771132,ind_recibo_ult1 ind_nom_pens_ult1 ind_nomina_u...,ind_recibo_ult1,ind_nom_pens_ult1,ind_nomina_ult1,ind_cno_fin_ult1,ind_tjcr_fin_ult1,ind_deco_fin_ult1,ind_dela_fin_ult1
680421,138599,ind_recibo_ult1 ind_tjcr_fin_ult1 ind_nom_pens...,ind_recibo_ult1,ind_tjcr_fin_ult1,ind_nom_pens_ult1,ind_nomina_ult1,ind_cno_fin_ult1,ind_dela_fin_ult1,ind_valo_fin_ult1
322648,1497817,ind_cco_fin_ult1 ind_ecue_fin_ult1 ind_recibo_...,ind_cco_fin_ult1,ind_ecue_fin_ult1,ind_recibo_ult1,ind_tjcr_fin_ult1,ind_cno_fin_ult1,ind_nom_pens_ult1,ind_nomina_ult1
321835,1500308,ind_nom_pens_ult1 ind_nomina_ult1 ind_cno_fin_...,ind_nom_pens_ult1,ind_nomina_ult1,ind_cno_fin_ult1,ind_tjcr_fin_ult1,ind_reca_fin_ult1,ind_ecue_fin_ult1,ind_ctop_fin_ult1
408667,1393371,ind_recibo_ult1 ind_tjcr_fin_ult1 ind_nom_pens...,ind_recibo_ult1,ind_tjcr_fin_ult1,ind_nom_pens_ult1,ind_dela_fin_ult1,ind_reca_fin_ult1,ind_nomina_ult1,ind_cno_fin_ult1


In [45]:
del bycus['added_products']

In [46]:
replacement_dict = {
    'ind_ahor_fin_ult1': 'green_bonds',
    'ind_aval_fin_ult1': 'insurance',
    'ind_cco_fin_ult1': 'business_household_loans',
    'ind_cder_fin_ult1': 'business_loans',
    'ind_cno_fin_ult1': 'mutual_funds',
    'ind_ctju_fin_ult1': 'credit_card',
    'ind_ctma_fin_ult1': 'bonds',
    'ind_ctop_fin_ult1': 'mortgage',
    'ind_ctpp_fin_ult1': 'saving_account',
    'ind_deco_fin_ult1': 'deposits',
    'ind_deme_fin_ult1': 'foreign_curency_exchange_services',
    'ind_dela_fin_ult1': 'long_term_deposits',
    'ind_ecue_fin_ult1': 'auto_loans',
    'ind_fond_fin_ult1': 'bill_pay_services',
    'ind_hip_fin_ult1': 'wealth_management',
    'ind_plan_fin_ult1': 'corporate_credit_cards',
    'ind_pres_fin_ult1': 'current_accounts',
    'ind_reca_fin_ult1': 'equipment_financing',
    'ind_tjcr_fin_ult1': 'stocks',
    'ind_valo_fin_ult1': 'Mobile_banking',
    'ind_viv_fin_ult1': 'student_loans',
    'ind_nomina_ult1': 'payroll_account',
    'ind_nom_pens_ult1': 'retirement_account',
    'ind_recibo_ult1': 'debit_cards'
}

bycus = bycus.replace(replacement_dict)

In [47]:
bycus.head()

,cust_id,product1,product2,product3,product4,product5,product6,product7
0,15889,debit_cards,retirement_account,payroll_account,equipment_financing,long_term_deposits,mutual_funds,auto_loans
1,1170544,debit_cards,retirement_account,payroll_account,mutual_funds,stocks,deposits,equipment_financing
2,1170545,debit_cards,retirement_account,payroll_account,mutual_funds,stocks,equipment_financing,auto_loans
3,1170547,debit_cards,retirement_account,payroll_account,mutual_funds,stocks,deposits,equipment_financing
4,1170548,debit_cards,retirement_account,payroll_account,mutual_funds,stocks,deposits,equipment_financing


### Recommend potential customers by product

In [48]:
prob.head()

,cust_id,green_bonds,insurance,business_household_loans,business_loans,mutual_funds,credit_card,bonds,mortgage,saving_account,deposits,foreign_curency_exchange_services,long_term_deposits,auto_loans,bill_pay_services,wealth_management,corporate_credit_cards,current_accounts,equipment_financing,stocks,Mobile_banking,student_loans,payroll_account,retirement_account,debit_cards
105,1170620.0,0.0,0.0,0.002194,3.227253e-10,0.011792,2.333352e-10,1.916583e-03,5.127091e-04,3.943911e-04,1.747549e-03,1.019134e-09,0.001505,0.004175,2.647928e-04,1.314085e-10,3.943938e-05,1.994857e-10,0.006105,0.006290,2.321230e-04,8.281709e-11,0.015085,0.015358,0.009593
671,1169651.0,0.0,0.0,0.025311,2.420173e-09,0.003889,4.716979e-09,7.734267e-08,6.265190e-08,3.940430e-08,1.152004e-07,7.974155e-09,0.001541,0.001575,6.159712e-08,8.588187e-10,4.906745e-09,1.833289e-09,0.000147,0.000074,7.704713e-04,7.916709e-10,0.002422,0.002569,0.004586
674,1169655.0,0.0,0.0,0.002701,2.313512e-10,0.008842,1.527281e-10,4.408597e-04,5.127078e-04,3.943900e-04,9.557721e-06,7.955852e-10,0.001041,0.002817,2.411451e-04,9.355075e-11,5.852877e-05,1.636344e-10,0.006880,0.008036,2.722963e-04,5.693747e-11,0.012812,0.015072,0.004734
1351,1172024.0,0.0,0.0,0.008114,1.220091e-09,0.004394,6.933238e-10,1.481134e-03,1.273039e-03,4.243593e-04,1.826733e-03,3.536713e-09,0.006829,0.002419,1.273039e-03,4.243383e-04,2.475172e-09,8.676054e-10,0.061421,0.069089,1.882500e-08,3.616346e-10,0.006419,0.006419,0.009431
1585,1171168.0,0.0,0.0,0.005300,8.574963e-10,0.001105,2.862840e-06,2.864903e-04,2.435558e-04,2.464144e-04,1.432963e-05,1.753735e-09,0.001080,0.003137,2.721778e-04,2.835813e-10,8.588126e-06,4.824185e-10,0.016142,0.012053,3.722021e-05,2.573847e-10,0.002591,0.002846,0.098161


In [49]:
prob.columns

Index(['cust_id', 'green_bonds', 'insurance', 'business_household_loans',
       'business_loans', 'mutual_funds', 'credit_card', 'bonds', 'mortgage',
       'saving_account', 'deposits', 'foreign_curency_exchange_services',
       'long_term_deposits', 'auto_loans', 'bill_pay_services',
       'wealth_management', 'corporate_credit_cards', 'current_accounts',
       'equipment_financing', 'stocks', 'Mobile_banking', 'student_loans',
       'payroll_account', 'retirement_account', 'debit_cards'],
      dtype='object')

In [50]:
sorted_prob3 = prob.sort_values(by=['business_household_loans'], ascending=False)
byproduct3 = sorted_prob3['cust_id'].iloc[:20]

In [51]:
sorted_prob4 = prob.sort_values(by=['business_loans'], ascending=False)
byproduct4 = sorted_prob4['cust_id'].iloc[:20]

In [52]:
sorted_prob5 = prob.sort_values(by=['mutual_funds'], ascending=False)
byproduct5 = sorted_prob5['cust_id'].iloc[:20]

In [53]:
sorted_prob6 = prob.sort_values(by=['credit_card'], ascending=False)
byproduct6 = sorted_prob6['cust_id'].iloc[:20]

In [54]:
sorted_prob7 = prob.sort_values(by=['bonds'], ascending=False)
byproduct7 = sorted_prob7['cust_id'].iloc[:20]

In [55]:
sorted_prob8 = prob.sort_values(by=['mortgage'], ascending=False)
byproduct8 = sorted_prob8['cust_id'].iloc[:20]

In [56]:
sorted_prob9 = prob.sort_values(by=['saving_account'], ascending=False)
byproduct9 = sorted_prob9['cust_id'].iloc[:20]

In [57]:
sorted_prob10 = prob.sort_values(by=['deposits'], ascending=False)
byproduct10 = sorted_prob10['cust_id'].iloc[:20]

In [58]:
sorted_prob11 = prob.sort_values(by=['foreign_curency_exchange_services'], ascending=False)
byproduct11 = sorted_prob11['cust_id'].iloc[:20]

In [59]:
sorted_prob12 = prob.sort_values(by=['long_term_deposits'], ascending=False)
byproduct12 = sorted_prob12['cust_id'].iloc[:20]

In [60]:
sorted_prob13 = prob.sort_values(by=['auto_loans'], ascending=False)
byproduct13 = sorted_prob13['cust_id'].iloc[:20]

In [61]:
sorted_prob14 = prob.sort_values(by=['bill_pay_services'], ascending=False)
byproduct14 = sorted_prob14['cust_id'].iloc[:20]

In [62]:
sorted_prob15 = prob.sort_values(by=['wealth_management'], ascending=False)
byproduct15 = sorted_prob15['cust_id'].iloc[:20]

In [63]:
sorted_prob16 = prob.sort_values(by=['corporate_credit_cards'], ascending=False)
byproduct16 = sorted_prob16['cust_id'].iloc[:20]

In [64]:
sorted_prob17 = prob.sort_values(by=['current_accounts'], ascending=False)
byproduct17 = sorted_prob17['cust_id'].iloc[:20]

In [65]:
sorted_prob18 = prob.sort_values(by=['equipment_financing'], ascending=False)
byproduct18 = sorted_prob18['cust_id'].iloc[:20]

In [66]:
sorted_prob19 = prob.sort_values(by=['stocks'], ascending=False)
byproduct19 = sorted_prob19['cust_id'].iloc[:20]

In [67]:
sorted_prob20 = prob.sort_values(by=['Mobile_banking'], ascending=False)
byproduct20 = sorted_prob20['cust_id'].iloc[:20]

In [68]:
sorted_prob21 = prob.sort_values(by=['student_loans'], ascending=False)
byproduct21 = sorted_prob21['cust_id'].iloc[:20]

In [69]:
sorted_prob22 = prob.sort_values(by=['payroll_account'], ascending=False)
byproduct22 = sorted_prob22['cust_id'].iloc[:20]

In [70]:
sorted_prob23 = prob.sort_values(by=['retirement_account'], ascending=False)
byproduct23 = sorted_prob23['cust_id'].iloc[:20]

In [71]:
sorted_prob24 = prob.sort_values(by=['debit_cards'], ascending=False)
byproduct24 = sorted_prob24['cust_id'].iloc[:20]

In [72]:
transposed3 = byproduct3.transpose()

In [73]:
transposed4 = byproduct4.transpose()
transposed5 = byproduct5.transpose()
transposed6 = byproduct6.transpose()
transposed7 = byproduct7.transpose()
transposed8 = byproduct8.transpose()
transposed9 = byproduct9.transpose()
transposed10 = byproduct10.transpose()
transposed11 = byproduct11.transpose()
transposed12 = byproduct12.transpose()
transposed13 = byproduct13.transpose()
transposed14 = byproduct14.transpose()
transposed15 = byproduct15.transpose()
transposed16 = byproduct16.transpose()
transposed17 = byproduct17.transpose()
transposed18 = byproduct18.transpose()
transposed19 = byproduct19.transpose()
transposed20 = byproduct20.transpose()
transposed21 = byproduct21.transpose()
transposed22 = byproduct22.transpose()
transposed23 = byproduct23.transpose()
transposed24 = byproduct24.transpose()

In [74]:
# Concatenate along rows (vertically)
transposed = pd.concat([transposed3, transposed4, transposed5, transposed6, transposed7, transposed8, transposed9, transposed10, transposed11, transposed12, transposed13, transposed14, transposed15, transposed16, transposed17, transposed18, transposed19, transposed20, transposed21, transposed22, transposed23, transposed24], ignore_index=True)

In [75]:
product_list = {
    'business_household_loans': ['business_household_loans'] * 20,
    'business_loans': ['business_loans'] * 20,
    'mutual_funds': ['mutual_funds'] * 20,
    'credit_card': ['credit_card'] * 20,
    'bonds': ['bonds'] * 20,
    'mortgage': ['mortgage'] * 20,
    'saving_account': ['saving_account'] * 20,
    'deposits': ['deposits'] * 20,
    'foreign_curency_exchange_services': ['foreign_curency_exchange_services'] * 20,
    'long_term_deposits': ['long_term_deposits'] * 20,
    'auto_loans': ['auto_loans'] * 20,
    'bill_pay_services': ['bill_pay_services'] * 20,
    'wealth_management': ['wealth_management'] * 20,
    'corporate_credit_cards': ['corporate_credit_cards'] * 20,
    'current_accounts': ['current_accounts'] * 20,
    'equipment_financing': ['equipment_financing'] * 20,
    'stocks': ['stocks'] * 20,
    'Mobile_banking': ['Mobile_banking'] * 20,
    'student_loans': ['student_loans'] * 20,
    'payroll_account': ['payroll_account'] * 20,
    'retirement_account': ['retirement_account'] * 20,
    'debit_cards': ['debit_cards'] * 20,
}
# Flatten the dictionary into a list
product_list_flat = [item for sublist in product_list.values() for item in sublist]

# Convert the Series to a DataFrame
transposed_df = transposed.to_frame(name='cust_id')  # You can give a name to the column

# Add the 'product' column to the DataFrame
transposed_df['product'] = product_list_flat

# If you want 'product' to be the first column, you can re-order the columns like this:
transposed_df = transposed_df[['product', 'cust_id']]

In [76]:
print(transposed_df)

                      product    cust_id
0    business_household_loans  1419737.0
1    business_household_loans  1403182.0
2    business_household_loans  1473025.0
3    business_household_loans  1527492.0
4    business_household_loans  1486874.0
..                        ...        ...
435               debit_cards  1519325.0
436               debit_cards  1401162.0
437               debit_cards  1242148.0
438               debit_cards  1062169.0
439               debit_cards  1391475.0

[440 rows x 2 columns]


In [77]:
# save to csv
transposed_df.to_csv('byproducts.csv',index=False)

### Probability to score

In [78]:
# Select only the columns with probabilities (excluding 'cust_id')
probabilities = prob.drop('cust_id', axis=1)

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the probabilities to scale them to the range [0, 1]
scaled_probabilities = scaler.fit_transform(probabilities)

# Create a new DataFrame with the scaled probabilities
# Make sure to pass the original DataFrame's index to the new DataFrame
prob_normalized = pd.DataFrame(scaled_probabilities, index=prob.index, columns=probabilities.columns)

# Add the 'cust_id' column back to the new DataFrame
# Since the indices now align, the 'cust_id' values will be correctly matched
prob_normalized.insert(0, 'cust_id', prob['cust_id'])

# Now 'prob_normalized' contains the normalized probabilities with the correct 'cust_id' values

In [79]:
prob.head()

,cust_id,green_bonds,insurance,business_household_loans,business_loans,mutual_funds,credit_card,bonds,mortgage,saving_account,deposits,foreign_curency_exchange_services,long_term_deposits,auto_loans,bill_pay_services,wealth_management,corporate_credit_cards,current_accounts,equipment_financing,stocks,Mobile_banking,student_loans,payroll_account,retirement_account,debit_cards
105,1170620.0,0.0,0.0,0.002194,3.227253e-10,0.011792,2.333352e-10,1.916583e-03,5.127091e-04,3.943911e-04,1.747549e-03,1.019134e-09,0.001505,0.004175,2.647928e-04,1.314085e-10,3.943938e-05,1.994857e-10,0.006105,0.006290,2.321230e-04,8.281709e-11,0.015085,0.015358,0.009593
671,1169651.0,0.0,0.0,0.025311,2.420173e-09,0.003889,4.716979e-09,7.734267e-08,6.265190e-08,3.940430e-08,1.152004e-07,7.974155e-09,0.001541,0.001575,6.159712e-08,8.588187e-10,4.906745e-09,1.833289e-09,0.000147,0.000074,7.704713e-04,7.916709e-10,0.002422,0.002569,0.004586
674,1169655.0,0.0,0.0,0.002701,2.313512e-10,0.008842,1.527281e-10,4.408597e-04,5.127078e-04,3.943900e-04,9.557721e-06,7.955852e-10,0.001041,0.002817,2.411451e-04,9.355075e-11,5.852877e-05,1.636344e-10,0.006880,0.008036,2.722963e-04,5.693747e-11,0.012812,0.015072,0.004734
1351,1172024.0,0.0,0.0,0.008114,1.220091e-09,0.004394,6.933238e-10,1.481134e-03,1.273039e-03,4.243593e-04,1.826733e-03,3.536713e-09,0.006829,0.002419,1.273039e-03,4.243383e-04,2.475172e-09,8.676054e-10,0.061421,0.069089,1.882500e-08,3.616346e-10,0.006419,0.006419,0.009431
1585,1171168.0,0.0,0.0,0.005300,8.574963e-10,0.001105,2.862840e-06,2.864903e-04,2.435558e-04,2.464144e-04,1.432963e-05,1.753735e-09,0.001080,0.003137,2.721778e-04,2.835813e-10,8.588126e-06,4.824185e-10,0.016142,0.012053,3.722021e-05,2.573847e-10,0.002591,0.002846,0.098161


In [80]:
prob_normalized.head()

,cust_id,green_bonds,insurance,business_household_loans,business_loans,mutual_funds,credit_card,bonds,mortgage,saving_account,deposits,foreign_curency_exchange_services,long_term_deposits,auto_loans,bill_pay_services,wealth_management,corporate_credit_cards,current_accounts,equipment_financing,stocks,Mobile_banking,student_loans,payroll_account,retirement_account,debit_cards
105,1170620.0,0.0,0.0,0.004859,6.076632e-09,0.014366,3.135160e-07,0.118166,1.894434e-03,9.947140e-04,0.102618,1.042547e-07,0.005568,0.010111,2.526237e-03,7.533121e-09,1.520730e-02,1.878850e-07,0.008178,0.023043,2.776534e-03,3.822236e-07,0.056199,0.019860,0.026845
671,1169651.0,0.0,0.0,0.056056,9.268663e-08,0.004738,7.016176e-06,0.000004,2.024840e-07,8.738396e-08,0.000006,1.821028e-06,0.005701,0.003813,5.117217e-07,8.639932e-08,1.719921e-06,1.829904e-06,0.000196,0.000271,9.216072e-03,3.912538e-06,0.009023,0.003321,0.012831
674,1169655.0,0.0,0.0,0.005983,2.303519e-09,0.010773,1.930155e-07,0.027180,1.894429e-03,9.947113e-04,0.000561,4.907393e-08,0.003850,0.006822,2.300621e-03,3.428565e-09,2.256800e-02,1.518534e-07,0.009216,0.029440,3.257073e-03,2.533349e-07,0.047732,0.019489,0.013247
1351,1172024.0,0.0,0.0,0.017970,4.313161e-08,0.005353,1.001159e-06,0.091318,4.703858e-03,1.070299e-03,0.107268,7.256925e-07,0.025263,0.005859,1.214563e-02,4.600697e-02,7.823279e-07,8.593643e-07,0.082275,0.253114,1.842284e-07,1.770822e-06,0.023913,0.008300,0.026390
1585,1171168.0,0.0,0.0,0.011739,2.815895e-08,0.001346,4.279662e-03,0.017663,8.999110e-04,6.214899e-04,0.000841,2.855832e-07,0.003995,0.007597,2.596695e-03,2.403178e-08,3.311333e-03,4.722405e-07,0.021622,0.044158,4.451744e-04,1.251625e-06,0.009650,0.003679,0.274726


### Add product had to test.csv

In [81]:
# Create the "pro1" column based on the condition
prob['pro1'] = '0'
prob['pro2'] = '0'
prob['pro3'] = np.where((prob['business_household_loans'] > 0) & (prob['business_household_loans'] < 1e-5), 1, 0)
prob['pro4'] = np.where((prob['business_loans'] > 0) & (prob['business_loans'] < 1e-9), 1, 0)
prob['pro5'] = np.where((prob['mutual_funds'] > 0) & (prob['mutual_funds'] < 1e-6), 1, 0)
prob['pro6'] = np.where((prob['credit_card'] > 0) & (prob['credit_card'] < 1e-10), 1, 0)
prob['pro7'] = np.where((prob['bonds'] > 0) & (prob['bonds'] < 1e-8), 1, 0)
prob['pro8'] = np.where((prob['mortgage'] > 0) & (prob['mortgage'] < 1e-7), 1, 0)
prob['pro9'] = np.where((prob['saving_account'] > 0) & (prob['saving_account'] < 1e-8), 1, 0)
prob['pro10'] = np.where((prob['deposits'] > 0) & (prob['deposits'] < 1e-8), 1, 0)
prob['pro11'] = np.where((prob['foreign_curency_exchange_services'] > 0) & (prob['foreign_curency_exchange_services'] < 1e-9), 1, 0)
prob['pro12'] = np.where((prob['long_term_deposits'] > 0) & (prob['long_term_deposits'] < 1e-7), 1, 0)
prob['pro13'] = np.where((prob['auto_loans'] > 0) & (prob['auto_loans'] < 1e-7), 1, 0)
prob['pro14'] = np.where((prob['bill_pay_services'] > 0) & (prob['bill_pay_services'] < 1e-8), 1, 0)
prob['pro15'] = np.where((prob['wealth_management'] > 0) & (prob['wealth_management'] < 1e-10), 1, 0)
prob['pro16'] = np.where((prob['corporate_credit_cards'] > 0) & (prob['corporate_credit_cards'] < 1e-9), 1, 0)
prob['pro17'] = np.where((prob['current_accounts'] > 0) & (prob['current_accounts'] < 1e-8), 1, 0)
prob['pro18'] = np.where((prob['equipment_financing'] > 0) & (prob['equipment_financing'] < 1e-6), 1, 0)
prob['pro19'] = np.where((prob['stocks'] > 0) & (prob['stocks'] < 1e-5), 1, 0)
prob['pro20'] = np.where((prob['Mobile_banking'] > 0) & (prob['Mobile_banking'] < 1e-7), 1, 0)
prob['pro21'] = np.where((prob['student_loans'] > 0) & (prob['student_loans'] < 1e-11), 1, 0)
prob['pro22'] = np.where((prob['payroll_account'] > 0) & (prob['payroll_account'] < 1e-6), 1, 0)
prob['pro23'] = np.where((prob['retirement_account'] > 0) & (prob['retirement_account'] < 1e-6), 1, 0)
prob['pro24'] = np.where((prob['debit_cards'] > 0) & (prob['debit_cards'] < 1e-4), 1, 0)

In [82]:
# get 24 columns from prob to test
selected_columns = ['pro1', 'pro2', 'pro3', 'pro4', 'pro5', 'pro6', 'pro7', 'pro8', 'pro9', 'pro10', 'pro11', 'pro12', 'pro13', 'pro14', 'pro15', 'pro16', 'pro17', 'pro18', 'pro19', 'pro20', 'pro21', 'pro22', 'pro23','pro24']
df_selected = prob[selected_columns].copy()
test = pd.concat([test, df_selected], axis=1)

### Save csv

In [83]:
# Change pro1 -pro24 columns names
test.rename(columns={
    'pro1': 'green_bonds',
    'pro2': 'insurance',
    'pro3': 'business_household_loans',
    'pro4': 'business_loans',
    'pro5': 'mutual_funds',
    'pro6': 'credit_card',
    'pro7': 'bonds',
    'pro8': 'mortgage',
    'pro9': 'saving_account',
    'pro10': 'deposits',
    'pro11': 'foreign_curency_exchange_services',
    'pro12': 'long_term_deposits',
    'pro13': 'auto_loans',
    'pro14': 'bill_pay_services',
    'pro15': 'wealth_management',
    'pro16': 'corporate_credit_cards',
    'pro17': 'current_accounts',
    'pro18': 'equipment_financing',
    'pro19': 'stocks',
    'pro20': 'Mobile_banking',
    'pro21': 'student_loans',
    'pro22': 'payroll_account',
    'pro23': 'retirement_account',
    'pro24': 'debit_cards'
}, inplace=True)

In [84]:
test.sample(10)

,fecha_dato,cust_id,emp_index,residence,sex,age,new_cust,seniority,is_primary,cust_type,cust_rel_type,residence_index,foreigner_index,channel,indfall,tipodom,province,province_name,active_index,income,segment,new_first_date,age_grouped,Occupation,Marital status,green_bonds,insurance,business_household_loans,business_loans,mutual_funds,credit_card,bonds,mortgage,saving_account,deposits,foreign_curency_exchange_services,long_term_deposits,auto_loans,bill_pay_services,wealth_management,corporate_credit_cards,current_accounts,equipment_financing,stocks,Mobile_banking,student_loans,payroll_account,retirement_account,debit_cards
311511,2016-06-28,1539672,N,VN,F,36,1,3,1,1.0,Active,Y,N,Open New Branches,N,1,28.0,Ho Chi Minh City,1,134087.870595,bachelor degree,2023-09-15,adult,athlete,Single,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
419933,2016-06-28,1411206,N,VN,F,23,0,11,1,1.0,Inactive,Y,N,Social Media Campaign,N,1,8.0,Hanoi,0,72860.400000,advanced degree,2023-01-22,adult,office worker,Single,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252572,2016-06-28,1483098,N,VN,M,21,0,8,1,1.0,Inactive,Y,N,Social Media Campaign,N,1,33.0,Bac Giang,0,134087.870595,advanced degree,2023-04-19,adult,corporations,Married,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
282531,2016-06-28,1450952,N,VN,M,21,0,9,1,1.0,Inactive,Y,N,Social Media Campaign,N,1,28.0,Ho Chi Minh City,0,62321.430000,advanced degree,2023-03-16,adult,scientist,Single,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
238510,2016-06-28,1468193,N,VN,M,20,0,9,1,1.0,Active,Y,N,Social Media Campaign,N,1,9.0,Son La,0,134087.870595,advanced degree,2023-04-02,young,lawyer,Married,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
307265,2016-06-28,1552611,N,VN,F,40,1,0,99,1.0,Active,Y,N,Open New Branches,N,1,10.0,Bac Ninh,0,134087.870595,bachelor degree,2023-11-25,adult,public official,Single,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
413217,2016-06-28,1378612,N,VN,M,60,0,17,1,1.0,Active,Y,N,Partnerships,N,1,28.0,Ho Chi Minh City,1,189974.370000,bachelor degree,2022-07-27,senior,pilot,Single,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
288449,2016-06-28,1441799,N,VN,M,20,0,10,1,1.0,Inactive,Y,N,Social Media Campaign,N,1,15.0,Dong Nai,1,134087.870595,advanced degree,2023-03-03,young,engineer,Single,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
421252,2016-06-28,1407228,N,VN,F,21,0,11,1,1.0,Inactive,Y,N,Social Media Campaign,N,1,39.0,Binh Dinh,0,100778.910000,advanced degree,2023-01-16,adult,journalist,Married,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
331551,2016-06-28,1512413,N,VN,F,9,0,7,1,1.0,Active,Y,N,Open New Branches,N,1,28.0,Ho Chi Minh City,1,134087.870595,bachelor degree,2023-06-02,young,chef,Married,0,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
# save to csv
bycus.to_csv('bycus.csv',index=False)

In [86]:
# save to csv
train.to_csv('train.csv',index=False)

In [87]:
# save to csv
test.to_csv('test.csv',index=False)

In [88]:
prob_normalized.to_csv('prob.csv',index=False)